In [ ]:

import os
import cv2
import matplotlib.pyplot as plt
import numpy as np


if not os.path.exists("ccl1.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/13_CCL/details.png --no-check-certificate

I_details = cv2.imread("details.png",cv2.IMREAD_GRAYSCALE)
plt.imshow(I_details)
plt.gray()
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
def Amf_phase_B(S,z_xy):
    if np.max(S) > z_xy > np.min(S):
        return z_xy
    else:
        return np.median(S)
def Amf_phase_A(img,x,y,min_convo,max_convo):
    convo = min_convo
    S = img[y-convo//2:y+convo//2+1,x-convo//2:x+convo//2+1]
    while True:
        if np.max(S) > np.median(S) > np.min(S):
            return Amf_phase_B(S,img[y,x])
        else:
            convo += 2
            S = img[y-convo//2:y+convo//2+1,x-convo//2:x+convo//2+1]
            if convo<=max_convo:
                continue
            else:
                return np.median(S)
                
def Adaptive_median_filtr(img, min_convo = 3 , max_convo = 7):
    return np.array([[ Amf_phase_A(img,j,i,min_convo,max_convo)  for j in range(max_convo//2,len(img[0])-max_convo//2)] for i in range(max_convo//2,len(img)-max_convo//2)])

In [ ]:
def segmentation2(img,x,y,th = 4):
    img = img.astype(float)
    visited = np.zeros(img.shape, dtype=bool)
    segmented = np.zeros(img.shape, dtype="uint8")
    stack = []
    stack.append((x,y))
    visited[x,y] = True
    segmented[x,y] = 1
    Sn = 0
    mV = img[x,y]
    while stack != []:
        x,y = stack[0]
        stack.pop(0)
        Sn +=1
        mV = (mV*(Sn-1) + img[x,y])/Sn
        for xi,yi in [(x+i,y+j) for i in [-1,0,1] for j in [-1,0,1] if ((i!=0 or j!=0) and (0<x+i<img.shape[0]-1) and (0<y+j<img.shape[1]-1))]:
            if not visited[xi,yi]:
                visited[xi,yi] = True
                d = abs(mV - img[xi,yi])
                if d<=th:
                    stack.append((xi,yi))
                    segmented[xi,yi] = 1
    return segmented

In [ ]:
I_details_adaptive = Adaptive_median_filtr(I_details, max_convo=7).astype('uint8')

In [ ]:

#I_details_BBHE = 255 - cv2.adaptiveThreshold(I_details_adaptive,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,4)
#I_details_BBHE = segmentation2(I_details_adaptive,122,222,70)
#I_details_adaptive_2 = Adaptive_median_filtr(I_details_BBHE, max_convo=7).astype('uint8')
#I_details_BBHE = cv2.morphologyEx(I_details_BBHE,cv2.MORPH_OPEN,np.ones((3,3)))
#I_details_BBHE = cv2.morphologyEx(I_details_BBHE,cv2.MORPH_OPEN,np.ones((3,3)))
I_details_BBHE = segmentation2(I_details_adaptive,122,222,80)
I = I_details_adaptive - I_details_adaptive * I_details_BBHE
I = (I>0)[1:-1,1:-1]
plt.imshow(I)
plt.gray()
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
tab = np.zeros(I.shape)
for i in range(len(I)):
    for j in range(len(I[i])):
        if I[i,j] == 1 and tab[i,j] != 1:
            tab = np.logical_or(segmentation2(I_details_adaptive[1:-1,1:-1],i,j,30),tab)
plt.imshow(tab)
plt.gray()
plt.xticks([])
plt.yticks([])
plt.show()
I_details_ad = 255 - cv2.adaptiveThreshold(I_details_adaptive,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,4)
I_ad = Adaptive_median_filtr(I_details_ad, max_convo=7).astype('uint8')
plt.imshow(I_details_ad)
plt.gray()
plt.xticks([])
plt.yticks([])
plt.show()
I = np.logical_or(I_details_ad[1:-1,1:-1],tab)
plt.imshow(I)
plt.gray()
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
#I_details_morph = cv2.morphologyEx(tab.astype('uint8'),cv2.MORPH_DILATE,np.ones((5,5)))

I_details_morph = cv2.morphologyEx(I.astype('uint8'),cv2.MORPH_CLOSE,np.ones((3,3)))
I_details_morph = cv2.morphologyEx(I_details_morph.astype('uint8'),cv2.MORPH_OPEN,np.ones((5,1)))
I_details_morph = cv2.morphologyEx(I_details_morph.astype('uint8'),cv2.MORPH_OPEN,np.ones((1,5)))
I_details_morph = cv2.morphologyEx(tab.astype('uint8'),cv2.MORPH_DILATE,np.ones((4,4)))

_,res = cv2.connectedComponents(I_details_morph)
plt.imshow(res)
plt.gray()
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
l,res = cv2.connectedComponents(I_details_morph)
ccl = 1
M = []
while ccl<l+1:
    I = (res == ccl)*1
    I = I.astype('uint8')
    moments = cv2.moments(I)
    Hu_moment = cv2.HuMoments(moments)
    M.append(Hu_moment.reshape(len(Hu_moment)))
    ccl+=1
M = np.array(M)
print(len(M))


In [ ]:

new = np.zeros(res.shape)
ccl = 1
while ccl<l+1:
    if M[ccl-1,3]>0.00015:
        new += (res == ccl)*1
    ccl+=1
plt.imshow(new)
plt.gray()
plt.xticks([])
plt.yticks([])
plt.show()


śrubki

In [ ]:

new = np.zeros(res.shape)
ccl = 1
while ccl<l+1:
    if np.logical_and(np.logical_and(M[ccl-1,3]<0.00015,np.logical_or(np.logical_and(M[ccl-1,1]>0.000031,M[ccl-1,6]<0.00000000000005),np.logical_and(M[ccl-1,3]>0.000000015,M[ccl-1,4]>0.0000000000015))),np.logical_or(M[ccl-1,2]<0.0000032,M[ccl-1,2]>0.0000033)):#M[ccl-1,5]>0.0000000000015
        new += (res == ccl)*1
    ccl+=1
plt.imshow(new)
plt.gray()
plt.xticks([])
plt.yticks([])
plt.show()


nakrętki

In [ ]:

new = np.zeros(res.shape)
ccl = 1
while ccl<l+1:
    if np.logical_and(np.logical_not(np.logical_and(np.logical_or(M[ccl-1,2]<0.0000032,M[ccl-1,2]>0.0000033),np.logical_or(np.logical_and(M[ccl-1,1]>0.000031,M[ccl-1,6]<0.00000000000005),np.logical_and(M[ccl-1,3]>0.000000015,M[ccl-1,4]>0.0000000000015)))),M[ccl-1,3]<0.00015):
        new += (res == ccl)*1
    ccl+=1
plt.imshow(new)
plt.gray()
plt.xticks([])
plt.yticks([])
plt.show()

nakładki